# MWC WEBPAGE SCRAPER
A scraper for the exhibitors webpage: https://www.mwcbarcelona.com/exhibition/2019-exhibitors/

In [15]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [16]:
class Exhibitor:
    def __init__(self, link):
        self.link = link
        self.name = link[39:-1]
        self.location = None
        self.description = None
        self.email = None
        self.phone = None
        self.website = None
    
    def __str__(self):
        return f'''
        Name: {self.name} 
        Location: {self.location}
        Description: {self.description[:50]}...
        Email: {self.email}
        Website: {self.website}
        '''
    def __repr__(self):
        return f'''
        Name: {self.name} 
        Location: {self.location}
        Description: {self.description[:50]}...
        Email: {self.email}
        Website: {self.website}
        Number: {self.phone}
        Link: {self.link}
        '''
    def to_list(self):
        return [self.name, self.link, self.location, self.phone, self.email, self.website, self.description]
        

In [29]:
def get_links(tipus):
    """This function acceses the main exhibitors webpage and gets a list with all the individual links
    
    Returns: list with individual links"""
    
    #list to store the links
    name_list = list()
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}

    for i in range(77):
        #request
        if tipus=='mwc':
            r = requests.get(f'https://www.mwcbarcelona.com/exhibition/2019-exhibitors/page/{i}', headers=headers)
        elif tipus=='4yfn':
            r = requests.get(f'https://www.4yfn.com/barcelona/2019-exhibitors/page/{i}', headers=headers)
        content = r.text
        
        #regex
        name_pattern = re.compile(r'https://www.mwcbarcelona.com/exhibitor/[-\w]+/')
        names = name_pattern.findall(content)

        name_list.extend(names)
    
    return name_list

In [30]:
def get_info(link):
    """Gets all the information in the page
    
    Params: link = It is the page you want to scrape
    Returns: Exhibitor object = An object with all the information"""
    
    #create object
    ex = Exhibitor(link)
    
    #request and prep
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    r = requests.get(link, headers=headers)
    content = r.content
    text = r.text
    soup = BeautifulSoup(content)
    
    #find location
    name_pattern = re.compile(r'Hall[\w\s]+')
    location = name_pattern.findall(text)
    if location:
        ex.location = location[0]
    
    #find description
    div_api_description = soup.find("div", {'class':'api-description'})
    if div_api_description:
        p_description = list(div_api_description.children)[1]
        description = list(p_description.children)
        if description:
            ex.description = description[0]
    
    #find phone
    p_contact_details = soup.find("p", string='Contact Details')
    if p_contact_details:
        parent_div = div_api_description.parent.parent
        search = re.search('\d+',parent_div.text)
        if search:
            phone = search.group()
    
    #find email
    email_pattern = re.compile(r'mailto:[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    email = email_pattern.findall(text)
    if email:
        ex.email = email[0][7:]
    
    #find website
    name_pattern = re.compile(r'class="web-site-link"\shref="http://www.[a-zA-Z0-9-.]+.[a-zA-Z0-9-.]+"')
    website = name_pattern.findall(str(soup))
    if website:
        ex.website = website[0][28:-1]
    
    return ex
    
    

In [31]:
def fetch_all(list_of_links, number=None):
    """Scrapes all the pages from list of links and returns a list with all the exhibitors"""
    info = list()
    i = 0
    if number!=None:
        links = list_of_links[0:number]
    else:
        links = list_of_links
    for link in links:
        print(link)
        i += 1
        if i%10 == 0:
            print(f"Dowloading exhibitor number {i}...")
        info.append(get_info(link))
    return info

In [32]:
def to_df(all_info):
    """Exports the list with all the exhibitors objects to a Dataframe""" 
    list_of_lists = [ex.to_list() for ex in all_info]
    df = pd.DataFrame(list_of_lists, columns = ['Name', 'Link', 'Location', 'Number', 'Email', 'Website', 'Description'])
    return df

In [33]:
links = get_links(tipus="mwc")

In [34]:
len(links)

3073

In [35]:
all_info = fetch_all(links)

https://www.mwcbarcelona.com/exhibitor/tappx/
https://www.mwcbarcelona.com/exhibitor/yet-gmbh/
https://www.mwcbarcelona.com/exhibitor/10xmotion/
https://www.mwcbarcelona.com/exhibitor/12cm-global-pte-ltd/
https://www.mwcbarcelona.com/exhibitor/1nce/
https://www.mwcbarcelona.com/exhibitor/1ot-2/
https://www.mwcbarcelona.com/exhibitor/1stpoint-hammer-communications/
https://www.mwcbarcelona.com/exhibitor/2hz/
https://www.mwcbarcelona.com/exhibitor/2operate-a-s/
https://www.mwcbarcelona.com/exhibitor/2sens/
Dowloading exhibitor number 10...
https://www.mwcbarcelona.com/exhibitor/360os/
https://www.mwcbarcelona.com/exhibitor/3dlook/
https://www.mwcbarcelona.com/exhibitor/3i-inc/
https://www.mwcbarcelona.com/exhibitor/3mk-screen-protectors/
https://www.mwcbarcelona.com/exhibitor/3starsnet/
https://www.mwcbarcelona.com/exhibitor/3z-telecom/
https://www.mwcbarcelona.com/exhibitor/42-maru-inc/
https://www.mwcbarcelona.com/exhibitor/4dreplay-korea-inc/
https://www.mwcbarcelona.com/exhibitor/4ii

https://www.mwcbarcelona.com/exhibitor/agc/
https://www.mwcbarcelona.com/exhibitor/ageify/
https://www.mwcbarcelona.com/exhibitor/agencia-catalana-de-turisme/
https://www.mwcbarcelona.com/exhibitor/agencia-de-desenvolupament-del-bergueda/
https://www.mwcbarcelona.com/exhibitor/agile-content/
https://www.mwcbarcelona.com/exhibitor/agm-group-limited/
https://www.mwcbarcelona.com/exhibitor/agora-io/
Dowloading exhibitor number 150...
https://www.mwcbarcelona.com/exhibitor/agora-io-2/
https://www.mwcbarcelona.com/exhibitor/agrandtech-limited/
https://www.mwcbarcelona.com/exhibitor/agrobit/
https://www.mwcbarcelona.com/exhibitor/agrocenta/
https://www.mwcbarcelona.com/exhibitor/agw-simulations/
https://www.mwcbarcelona.com/exhibitor/agw-simulations-2/
https://www.mwcbarcelona.com/exhibitor/ahqlab-inc/
https://www.mwcbarcelona.com/exhibitor/aictx-ag/
https://www.mwcbarcelona.com/exhibitor/aifa-technology-corp/
https://www.mwcbarcelona.com/exhibitor/aione-lifestyle-studios-inc/
Dowloading exh

https://www.mwcbarcelona.com/exhibitor/arjo-solutions/
https://www.mwcbarcelona.com/exhibitor/arm/
https://www.mwcbarcelona.com/exhibitor/arobs-transilvania-software/
https://www.mwcbarcelona.com/exhibitor/arroe-limited/
https://www.mwcbarcelona.com/exhibitor/art-map/
https://www.mwcbarcelona.com/exhibitor/art-fi/
https://www.mwcbarcelona.com/exhibitor/artda/
https://www.mwcbarcelona.com/exhibitor/artech-co-ltd/
https://www.mwcbarcelona.com/exhibitor/artesyn-embedded-technologies/
Dowloading exhibitor number 290...
https://www.mwcbarcelona.com/exhibitor/arthur-holm/
https://www.mwcbarcelona.com/exhibitor/artiza-networks/
https://www.mwcbarcelona.com/exhibitor/artlog-software-zone-srl-d/
https://www.mwcbarcelona.com/exhibitor/arvizio/
https://www.mwcbarcelona.com/exhibitor/as-catalana-cluster-de-la-industria-de-lesport-indescat/
https://www.mwcbarcelona.com/exhibitor/asavie/
https://www.mwcbarcelona.com/exhibitor/aselsan/
https://www.mwcbarcelona.com/exhibitor/asignio/
https://www.mwcba

https://www.mwcbarcelona.com/exhibitor/bird/
https://www.mwcbarcelona.com/exhibitor/birla-cable-limited/
https://www.mwcbarcelona.com/exhibitor/bisdn-gmbh/
https://www.mwcbarcelona.com/exhibitor/bitphy/
https://www.mwcbarcelona.com/exhibitor/bits/
https://www.mwcbarcelona.com/exhibitor/blackbelt-smartphone-defene/
https://www.mwcbarcelona.com/exhibitor/blackberry-limited/
https://www.mwcbarcelona.com/exhibitor/blackberry-mobile/
https://www.mwcbarcelona.com/exhibitor/blackbox-platform-ltd/
https://www.mwcbarcelona.com/exhibitor/blacknut/
Dowloading exhibitor number 430...
https://www.mwcbarcelona.com/exhibitor/blancco/
https://www.mwcbarcelona.com/exhibitor/blue-danube-systems/
https://www.mwcbarcelona.com/exhibitor/bluebank-communication-technology-co-ltd/
https://www.mwcbarcelona.com/exhibitor/bluecommunication/
https://www.mwcbarcelona.com/exhibitor/bluetium/
https://www.mwcbarcelona.com/exhibitor/bmb/
https://www.mwcbarcelona.com/exhibitor/bmw-ag/
https://www.mwcbarcelona.com/exhib

https://www.mwcbarcelona.com/exhibitor/chengdu-nts-technology-co-ltd/
https://www.mwcbarcelona.com/exhibitor/chiara/
https://www.mwcbarcelona.com/exhibitor/chilmyeongbio-co-ltd/
https://www.mwcbarcelona.com/exhibitor/china-communications-consultants-sl/
https://www.mwcbarcelona.com/exhibitor/china-mobile-limited/
https://www.mwcbarcelona.com/exhibitor/chiper/
https://www.mwcbarcelona.com/exhibitor/chkout-me/
https://www.mwcbarcelona.com/exhibitor/chunghwa-telecom/
https://www.mwcbarcelona.com/exhibitor/ciclogreen-move-and-win-s-l/
https://www.mwcbarcelona.com/exhibitor/ciena/
Dowloading exhibitor number 570...
https://www.mwcbarcelona.com/exhibitor/cigo/
https://www.mwcbarcelona.com/exhibitor/cirrus-logic/
https://www.mwcbarcelona.com/exhibitor/cis-groups-latin-america/
https://www.mwcbarcelona.com/exhibitor/cisco/
https://www.mwcbarcelona.com/exhibitor/citi-group/
https://www.mwcbarcelona.com/exhibitor/city-of-gelsenkirchen/
https://www.mwcbarcelona.com/exhibitor/city-of-the-hague/
ht

...

https://www.mwcbarcelona.com/exhibitor/sikur/
https://www.mwcbarcelona.com/exhibitor/silicom-ltd-connectivity-solutions/
https://www.mwcbarcelona.com/exhibitor/silicon-mitus/
https://www.mwcbarcelona.com/exhibitor/silver-peak/
https://www.mwcbarcelona.com/exhibitor/simcar/
https://www.mwcbarcelona.com/exhibitor/simcom-wireless-solutions-co-ltd/
https://www.mwcbarcelona.com/exhibitor/simgas/
https://www.mwcbarcelona.com/exhibitor/sinch/
Dowloading exhibitor number 2470...
https://www.mwcbarcelona.com/exhibitor/singerhood/
https://www.mwcbarcelona.com/exhibitor/singular/
https://www.mwcbarcelona.com/exhibitor/singularbrain-consulting-s-l/
https://www.mwcbarcelona.com/exhibitor/sinocastel-co-ltd/
https://www.mwcbarcelona.com/exhibitor/siradel/
https://www.mwcbarcelona.com/exhibitor/sirin-labs/
https://www.mwcbarcelona.com/exhibitor/sisvel-group/
https://www.mwcbarcelona.com/exhibitor/sit-consulting-sl/
https://www.mwcbarcelona.com/exhibitor/siteng-electronic-co-ltd/
https://www.mwcbarcelo

https://www.mwcbarcelona.com/exhibitor/sudpay-sarl/
https://www.mwcbarcelona.com/exhibitor/suiff/
https://www.mwcbarcelona.com/exhibitor/sumec-wasin-telecom-co-ltd/
https://www.mwcbarcelona.com/exhibitor/summa-networks/
https://www.mwcbarcelona.com/exhibitor/summit-tech/
https://www.mwcbarcelona.com/exhibitor/summview/
https://www.mwcbarcelona.com/exhibitor/sunculture/
https://www.mwcbarcelona.com/exhibitor/sunnyway/
https://www.mwcbarcelona.com/exhibitor/suntech/
https://www.mwcbarcelona.com/exhibitor/sunwave-solutions/
Dowloading exhibitor number 2610...
https://www.mwcbarcelona.com/exhibitor/super-micro-computer-b-v/
https://www.mwcbarcelona.com/exhibitor/superbin/
https://www.mwcbarcelona.com/exhibitor/superiors/
https://www.mwcbarcelona.com/exhibitor/suprema/
https://www.mwcbarcelona.com/exhibitor/surcle/
https://www.mwcbarcelona.com/exhibitor/surfsight/
https://www.mwcbarcelona.com/exhibitor/suzhou-alton-electronics-technology-co-ltd/
https://www.mwcbarcelona.com/exhibitor/suzhou

https://www.mwcbarcelona.com/exhibitor/tempow/
https://www.mwcbarcelona.com/exhibitor/teoco/
https://www.mwcbarcelona.com/exhibitor/tera-srl/
https://www.mwcbarcelona.com/exhibitor/tera-technos/
https://www.mwcbarcelona.com/exhibitor/tesla-spain-slu/
https://www.mwcbarcelona.com/exhibitor/tessares/
https://www.mwcbarcelona.com/exhibitor/testbirds-gmbh/
Dowloading exhibitor number 2740...
https://www.mwcbarcelona.com/exhibitor/testcard-ltd/
https://www.mwcbarcelona.com/exhibitor/testfyra/
https://www.mwcbarcelona.com/exhibitor/testilabs/
https://www.mwcbarcelona.com/exhibitor/tethir-ltd/
https://www.mwcbarcelona.com/exhibitor/tetragon-sa/
https://www.mwcbarcelona.com/exhibitor/texel-2/
https://www.mwcbarcelona.com/exhibitor/texel/
https://www.mwcbarcelona.com/exhibitor/the-entertainer-fz-llc/
https://www.mwcbarcelona.com/exhibitor/the-evans-hunt-group/
https://www.mwcbarcelona.com/exhibitor/the-hague-business-agency/
Dowloading exhibitor number 2750...
https://www.mwcbarcelona.com/exhib

https://www.mwcbarcelona.com/exhibitor/vanguard-industries-inc/
https://www.mwcbarcelona.com/exhibitor/vanrise-solutions/
https://www.mwcbarcelona.com/exhibitor/varari/
https://www.mwcbarcelona.com/exhibitor/varjo-technologies/
https://www.mwcbarcelona.com/exhibitor/vas-creative-co-ltd/
Dowloading exhibitor number 2880...
https://www.mwcbarcelona.com/exhibitor/vayyar/
https://www.mwcbarcelona.com/exhibitor/vbox/
https://www.mwcbarcelona.com/exhibitor/veea/
https://www.mwcbarcelona.com/exhibitor/veo/
https://www.mwcbarcelona.com/exhibitor/veon/
https://www.mwcbarcelona.com/exhibitor/veridic-technologies-pvt-ltd/
https://www.mwcbarcelona.com/exhibitor/verimatrix-inc/
https://www.mwcbarcelona.com/exhibitor/veriport-information-technologies/
https://www.mwcbarcelona.com/exhibitor/verisoft/
https://www.mwcbarcelona.com/exhibitor/veritran/
Dowloading exhibitor number 2890...
https://www.mwcbarcelona.com/exhibitor/verizon/
https://www.mwcbarcelona.com/exhibitor/verizon-media/
https://www.mwcb

https://www.mwcbarcelona.com/exhibitor/xiamen-fourier-electronic-co-ltd/
https://www.mwcbarcelona.com/exhibitor/xiamen-honlly-electronic-technology-co-ltd/
https://www.mwcbarcelona.com/exhibitor/xiaomi-corporation/
https://www.mwcbarcelona.com/exhibitor/xignsys-gmbh/
https://www.mwcbarcelona.com/exhibitor/xilinx-ltd/
https://www.mwcbarcelona.com/exhibitor/xilinx-inc/
Dowloading exhibitor number 3020...
https://www.mwcbarcelona.com/exhibitor/xing-chuang-wei-ye-electronic-technology-sz-co-ltd/
https://www.mwcbarcelona.com/exhibitor/xmos/
https://www.mwcbarcelona.com/exhibitor/xperi/
https://www.mwcbarcelona.com/exhibitor/xpertbilling/
https://www.mwcbarcelona.com/exhibitor/xpik/
https://www.mwcbarcelona.com/exhibitor/xplora-technologies-as/
https://www.mwcbarcelona.com/exhibitor/xs-infosol-pvt-ltd/
https://www.mwcbarcelona.com/exhibitor/xspot/
https://www.mwcbarcelona.com/exhibitor/xu-hong-healthcare-co-ltd-brand-health-inventor/
https://www.mwcbarcelona.com/exhibitor/y-tech/
Dowloading 

In [36]:
df = to_df(all_info)

In [37]:
df.to_csv("./exhibitors_mwc.csv")

## Send EMAIL

In [313]:
import smtplib
from email.mime.text import MIMEText
import cred

def send_emails(all_info, user, password):
    i = 0
    #Acces credentials
    gmail_user = user 
    gmail_password = password


    #Server set up
    server = smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    server.ehlo()
    server.login(gmail_user, gmail_password)
    
    #send_email(server, 'jaume', 'jaumecolomhernandez@gmail.com')
    
    #Send email
    for ex in all_info:
        if ex.email!=None:
            time.sleep(1)
            send_email(server, ex.name, ex.email)
            print(f"Email sent to: {ex.name}")
            i += 1
            if i%10 == 0: print(f"{i}th email sent")
    print(f"Finished -- {i} emails sent")

    server.quit()


In [314]:
def send_email(server, name, email):
    email_text = f"""
Dear {name.capitalize()},

My name is Jaume Colom, I am studying at the technical university of Catalonia and part of the ForumTIC organizing team, a fair in which the best tech companies get in touch with computer science and telecommunications students.

I am writing you because next week starts the MWC and I would really like to assist. Especially because I think that it is a great place to meet with companies and learn. That is why I would like to know whether it is possible to get a ticket for the MWC. Thanks for your time and your attention.

Best regards,

Jaume
"""
    msg = MIMEText(email_text)
    msg['Subject'] = 'MWC Contact'
    msg['From'] = 'jaume.colom@forumtic.upc.edu'
    msg['To'] = email
    
    server.sendmail('jaume.colom@forumtic.upc.edu', email, msg.as_string())

In [316]:
send_emails(all_info[344:], cred.gmail_user, cred.gmail_password)

Email sent to: fifth9
Email sent to: filmclub
Email sent to: filtronic
Email sent to: finboot
Email sent to: flexvpc
Email sent to: fluo
Email sent to: focalspec
Email sent to: folan
Email sent to: followhealth
Email sent to: forcemanager
10th email sent
Email sent to: formind
Email sent to: forsk
Email sent to: fortumo
Email sent to: foru
Email sent to: fundeen
Email sent to: furgo
Email sent to: futuredone
Email sent to: fyber
Email sent to: galgus
Email sent to: galtronics
20th email sent
Email sent to: garmin
Email sent to: gemalto
Email sent to: gencell
Email sent to: geoedge
Email sent to: gigabyte
Email sent to: glassbox
Email sent to: glo
Email sent to: globalfoundries
Email sent to: globaltel
Email sent to: globitel
30th email sent
Email sent to: gmelius
Email sent to: gmv
Email sent to: gocase
Email sent to: golumi
Email sent to: gomapi
Email sent to: gomspace
Email sent to: goodbarber
Email sent to: goui
Email sent to: goyou
Email sent to: gradiant
40th email sent
Email sent

Email sent to: sikiwis
Email sent to: sikur
Email sent to: simcar
330th email sent
Email sent to: singerhood
Email sent to: smaaart
Email sent to: smaato
Email sent to: smanos
Email sent to: smartviser
Email sent to: snowfire
Email sent to: soccerdream
Email sent to: sofrecom
Email sent to: softech
Email sent to: softex
340th email sent
Email sent to: solace
Email sent to: solve
Email sent to: somnox
Email sent to: soundpays
Email sent to: sowhen
Email sent to: spaceboost
Email sent to: spectronite
Email sent to: spitche
Email sent to: sprint
Email sent to: squadata
350th email sent
Email sent to: stageinhome
Email sent to: starchium
Email sent to: startupxplore
Email sent to: stmicroelectronics
Email sent to: storytel
Email sent to: stratioinc
Email sent to: streamwide
Email sent to: strongkey
Email sent to: subex
Email sent to: subtonomy
360th email sent
Email sent to: suiff
Email sent to: summview
Email sent to: suntech
Email sent to: superiors
Email sent to: suprema
Email sent to: 